In [14]:
from  lib.mongo_db  import db_conn
import pandas as pd
import time
import numpy as np

In [64]:
def get_data(begin_date,end_date):
    data=db_conn.sid_info.find({"end_time":{'$gte':begin_date,'$lte':end_date},'status':0,'crawler_channel':{'$exists':False}})
    data=pd.DataFrame(line for line in data)
    return data

In [65]:
def anly_month_loss(begin_date,end_date):
    
    data=get_data(begin_date,end_date)
    data['one_month_loss_len']=data['call_log_missing_month_list'].apply(lambda x:len(x))

    _key_codes = data['one_month_loss_len'].values
    grp1=data['sid'].groupby(_key_codes)
    cnt1 = grp1.agg(np.size)
    ret_list=[]
    for i in range(6):
        ret_list.append([i,cnt1[i]])

    ret=pd.DataFrame(np.array(ret_list),columns=['整月缺失数','计数'])
    one_day_date=time.strftime("%Y-%m-%d", time.localtime(begin_date))
    print(one_day_date)
    ret['日期']=one_day_date
    return ret.ix[:,['日期','整月缺失数','计数']]
    

In [70]:
def anly_month_has_loss(begin_date,end_date):
    
    data=get_data(begin_date,end_date)
    def comp(a,b,c):
        return len(a)+len(b)+len(c)
    data['month_has_loss_len']=data.apply(lambda x:comp(x['call_log_missing_month_list'],x['call_log_possibly_missing_month_list'],x['call_log_part_missing_month_list']),axis=1)


    _key_codes = data['month_has_loss_len'].values
    grp1=data['sid'].groupby(_key_codes)
    cnt1 = grp1.agg(np.size)
    ret_list=[]
    for i in range(cnt1.shape[0]):
        ret_list.append([i,cnt1[i]])

    ret=pd.DataFrame(np.array(ret_list),columns=['有缺失月份数','计数'])
    one_day_date=time.strftime("%Y-%m-%d", time.localtime(begin_date))
    print(one_day_date)
    ret['日期']=one_day_date
    return ret.ix[:,['日期','有缺失月份数','计数']]

In [71]:
def month_all_loss_date():
    begin_date='20180806'
    timeArray = time.strptime(begin_date, "%Y%m%d")
    begin_date = time.mktime(timeArray)
    all_pd=pd.DataFrame(columns=['日期','整月缺失数','计数'])
    has_pd=pd.DataFrame(columns=['日期','有缺失月份数','计数'])
    while(1):
        if begin_date>=1536213774:
            break
        end_date=begin_date+86400
        all_ret=anly_month_loss(begin_date,end_date)
        has_ret=anly_month_has_loss(begin_date,end_date)
        all_pd=pd.concat([all_pd,all_ret])
        has_pd=pd.concat([has_pd,has_ret])
        begin_date=end_date
    all_pd.to_csv('整月缺失数统计.csv',index= False)
    has_pd.to_csv('有缺失月份数统计.csv',index= False)


In [72]:
month_all_loss_date()

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  app.launch_new_instance()
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:19: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
